In [7]:
import appdirs
import os

user_profile = os.path.join(appdirs.user_data_dir(), "Microsoft", "Edge", "User Data")
cookies = os.path.join(user_profile, "Default", "Network", "Cookies")
cookies_copy = r"C:\Users\BRARAUJONA1\Documents\GitHub\nsc\private\Cookies"

In [ ]:
import shutil

shutil.copy(cookies, cookies_copy)

In [20]:
import sqlite3

db = sqlite3.connect(cookies_copy)
# ignore decoding errors
db.text_factory = lambda b: b.decode(errors="ignore")
cursor = db.cursor()
# get the cookies from `cookies` table
cursor.execute("""
SELECT host_key, name, value, creation_utc, last_access_utc, expires_utc, encrypted_value 
FROM cookies""")

cookies_unfiltered = cursor.fetchall()

In [17]:
import json
import win32crypt
import base64

local_state = os.path.join(user_profile, "Local State")

with open(local_state, "r", encoding="utf-8") as f:
        local_state_dict = f.read()
        local_state_dict = json.loads(local_state_dict)

key = base64.b64decode(local_state_dict["os_crypt"]["encrypted_key"])
key = key[5:]
encryption_key = win32crypt.CryptUnprotectData(key, None, None, None, 0)[1]

In [18]:
from Crypto.Cipher import AES

def decrypt_data(data, key):
    try:
        # get the initialization vector
        iv = data[3:15]
        data = data[15:]
        # generate cipher
        cipher = AES.new(key, AES.MODE_GCM, iv)
        # decrypt password
        return cipher.decrypt(data)[:-16].decode()
    except:
        try:
            return str(win32crypt.CryptUnprotectData(data, None, None, None, 0)[1])
        except:
            # not supported
            return ""

In [21]:
cookies_list = []

for host_key, name, value, creation_utc, last_access_utc, expires_utc, encrypted_value in cookies_unfiltered:
    if not value:
        value = decrypt_data(encrypted_value, encryption_key)
    cookies_list.append(
        {
            "domain": host_key,
            "name": name,
            "value": value,
            "expiry": expires_utc
        }
    )